In [1890]:
from game import Game
import time
import os
import re
from collections import deque


## Load maps

In [1891]:
def extract_map_files(directory):
    pattern = re.compile(r'^map(\d+)\.txt$')
    map_file_indices = []

    for file_name in os.listdir(directory):
        match = pattern.match(file_name)
        if match:
            map_file_indices.append(match.group(1))

    return [int(idx) for idx in map_file_indices]

def is_valid_input(map, indices, algorithm, solvers):
    valid_input = True
    if map not in indices:
        print(f"Map index out of range. Please choose within range {min(indices)} to {max(indices)}")
        valid_input = False
    if algorithm not in solvers.keys():    
        print(f"{algorithm} is not a defined algorithm. Please choose from", *[f"{solver} ({i+1})  " for i, solver in enumerate(solvers.keys())])
        valid_input = False
    return valid_input

def load_map(map_index):  
    file_name = "map" + str(map_index) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    return game_map

map_file_indices = extract_map_files("./assets/maps/")

## Tutorial

In [1892]:
print("This is an example of the game map:")
map = load_map(2)
game = Game(map)
game.display_map()

This is an example of the game map:
W	P1	H	W	W	W	W
W	W	W	G1	W	W	W
W	W	W	B1	W	W	W
W	G2	B2	.	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [1893]:
game.get_box_locations()

[(2, 3), (3, 2), (4, 3)]

In [1894]:
game.get_goal_locations()

[(1, 3), (3, 1), (5, 3)]

In [1895]:
game.get_player_position()

(0, 2)

- W : Wall
- H : Human
- B : Box
- P : Portal
- G : Goal

In [1896]:
for direction in ['U', 'D', 'R', 'L']:
    result = game.apply_move(direction)
    print(f"Move {direction} is valid: {result}")
    if result:
        game.display_map()

Move U is valid: False
Move D is valid: False
Move R is valid: False
Move L is valid: True
W	P1	.	W	W	W	W
W	W	W	G1	W	W	W
W	W	W	B1	W	W	W
W	G2	B2	H	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [1897]:
game.apply_move('U')
game.display_map()

W	P1	.	W	W	W	W
W	W	W	G1/B1	W	W	W
W	W	W	H	W	W	W
W	G2	B2	.	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [1898]:
game.apply_moves(['D', 'L', 'R', 'D']) 
game.display_map()
print("Is game won?", game.is_game_won())

W	P1	.	W	W	W	W
W	W	W	G1/B1	W	W	W
W	W	W	.	W	W	W
W	G2/B2	.	.	P1	W	W
W	W	W	H	W	W	W
W	W	W	G3/B3	W	W	W
W	W	W	W	W	W	W
Is game won? True


## Solvers

### BFS

## Breadth-First Search (BFS) Implementation Explanation
In our BFS implementation, we systematically explore all possible game states level by level to find the shortest sequence of moves that leads to a solution.

🔹 What Does BFS Do?
Initialize the game state:

We extract the initial position of the player and all box positions from the given game_map.
We use a queue (deque) to store the game states, where each state consists of:
The player's position
The current locations of all boxes
The sequence of moves taken to reach this state
Start BFS Search:

We begin with the initial state and mark it as visited to avoid re-exploring the same state.
The visited_count variable tracks how many unique states have been explored.
Explore the State Space Level by Level:

While the queue is not empty, we dequeue the oldest state (FIFO order).
We check if the current game state is a winning state (all boxes are on goal positions).
If it’s a winning state, we return the shortest path to reach the solution.
Generate Next Possible Moves:

For each direction (Up, Down, Left, Right), we try to move the player.
If the move is valid (apply_move(direction) returns True):
We generate the new game state.
If this new state has not been visited before, we add it to the queue.
The move sequence is updated accordingly.
End Condition:

If all possibilities are explored and no solution is found, we return "No solution" along with the number of visited states.

🔹 Why Use BFS?
✔ Guarantees the shortest solution (because BFS explores all states level by level).
✔ Systematically explores all possibilities and prevents getting stuck in loops.
✔ Ensures completeness (if a solution exists, BFS will find it).

🔹 Key Components in Code
Code Section	Purpose
queue = deque([(initial_state, "")])	Stores game states to explore
visited = {initial_state}	Keeps track of visited states to avoid loops
queue.popleft()	Removes the oldest state for exploration (FIFO order)
for direction in ['U', 'D', 'L', 'R']:	Generates all possible moves
if new_state not in visited:	Ensures we don’t revisit the same state


In [1899]:
from collections import deque

def solver_bfs(game_map):
    game = Game(game_map)
    initial_state = (game.get_player_position(), tuple(game.get_box_locations()))

    queue = deque([(initial_state, "")])
    visited = {initial_state}
    visited_count = 0

    while queue:
        (player_pos, boxes), path = queue.popleft()
        visited_count += 1
        
        temp_game = Game(game_map)
        temp_game.set_player_position(player_pos)
        temp_game.set_box_positions(boxes)

        if temp_game.is_game_won():
            return path, visited_count  
        
        for direction in ['U', 'D', 'L', 'R']:
            new_game = Game(game_map)
            new_game.set_player_position(player_pos)
            new_game.set_box_positions(boxes)

            if new_game.apply_move(direction): 
                new_state = (new_game.get_player_position(), tuple(new_game.get_box_locations()))

                if new_state not in visited:
                    visited.add(new_state)
                    queue.append((new_state, path + direction))

    return "No solution", visited_count 


### DFS

### Depth-First Search (DFS) Implementation Explanation
In our DFS implementation, we explore the game state space by going as deep as possible along each path before backtracking. This approach differs from BFS, which explores all states at the same depth level first.

🔹 What Does DFS Do?
Initialize the game state:

Extract the initial player position and box locations from the game_map.
Use a stack (LIFO order) to store the game states, where each state includes:
The player's position
The locations of all boxes
The path taken to reach this state
The current search depth
The number of visited states
Start DFS Search:

Begin with the initial state and push it onto the stack.
Use a visited set to keep track of explored states and avoid loops.
Explore the State Space Depth-First:

While the stack is not empty, pop the most recent state.
Check if all boxes are on goal positions (winning state).
If a solution is found, return the shortest move sequence.
Backtracking & Depth Limitation:

If the current depth exceeds max_depth, the search stops at this branch and backtracks.
If the state has already been visited, it is skipped.
Generate Next Possible Moves:

Try moving the player Up, Down, Left, and Right.
If the move is valid (apply_move(direction) returns True):
Generate the new game state.
Push the new state onto the stack with an increased depth.
The move sequence is updated accordingly.
End Condition:

If all possibilities are explored and no solution is found, return "No solution" along with the number of visited states.
🔹 Why Use DFS?
✔ Good for large search spaces when the solution is deep.
✔ Consumes less memory than BFS since it doesn’t store all states at the same level.
✔ Can be optimized using iterative deepening (IDDFS) to prevent excessive depth search.

❌ However, DFS is not guaranteed to find the shortest solution since it may go deep into an inefficient path before backtracking.

🔹 Key Components in Code
Code Section	Purpose
stack = [(initial_state, "", 0, 0)]	Stores game states to explore (LIFO)
visited = set()	Keeps track of visited states to avoid loops
stack.pop()	Removes the most recent state for exploration (LIFO order)
for direction in ['U', 'D', 'L', 'R']:	Generates all possible moves
if depth > max_depth:	Prevents DFS from going too deep
stack.append((new_state, path + direction, depth + 1, visited_count + 1))	Pushes new valid states onto the stack

In [1900]:
from collections import deque

def solver_dfs(game_map, max_depth=50):
    game = Game(game_map)
    initial_state = (game.get_player_position(), tuple(game.get_box_locations()))
    goals = set(game.get_goal_locations())
    
    stack = [(initial_state, "", 0, 0)]  
    visited = set()
    
    while stack:
        (player_pos, boxes), path, depth, visited_count = stack.pop()

        if set(boxes) == goals:
            return path, visited_count
        if depth > max_depth:
            continue
        
        state = (player_pos, boxes)
        if state in visited:
            continue
        visited.add(state)
        for direction in ['U', 'D', 'L', 'R']:
            new_game = Game(game_map)
            new_game.set_player_position(player_pos)
            new_game.set_box_positions(boxes)

            if new_game.apply_move(direction):
                new_state = (new_game.get_player_position(), tuple(new_game.get_box_locations()))
                stack.append((new_state, path + direction, depth + 1, visited_count + 1))
    
    return "No solution", len(visited)

### IDS

### Iterative Deepening Search (IDS) Implementation Explanation
The Iterative Deepening Search (IDS) algorithm combines the advantages of Depth-First Search (DFS) and Breadth-First Search (BFS). It repeatedly runs DFS with an increasing depth limit, ensuring that if a solution exists, it finds the shortest path while keeping memory usage low.

🔹 What Does IDS Do?
Initialize the Game State:

Extract the initial player position and box locations from game_map.
Start with a depth limit of 0 and increase it iteratively.
Maintain a visited set to track explored states.
Run Depth-Limited DFS:

Perform a depth-limited DFS (DLS) where we only explore paths up to the current depth limit.
If the game is won within the depth limit, return the shortest path.
If the depth limit is reached without finding a solution, stop exploration and return None.
Iteratively Increase Depth:

If no solution is found at the current depth limit, increase the depth and restart DFS.
This process continues until a solution is found or all possibilities are explored.
Backtracking:

Since DFS is used at each depth, the algorithm backtracks when a move is not leading to a solution.
This prevents unnecessary exploration of paths that don’t contribute to solving the problem.
End Condition:

If a valid solution is found, return the move sequence and the number of visited states.
Otherwise, keep increasing the depth until no further expansion is possible.
🔹 Why Use IDS?
✔ Finds the shortest solution (like BFS) by incrementally deepening.
✔ Uses less memory compared to BFS because each DFS only keeps track of the current path.
✔ Avoids the inefficiencies of DFS by ensuring shallow solutions are found first.

❌ However, IDS is computationally expensive, as it repeats DFS from the start at each depth level.

🔹 Key Components in Code
Code Section	Purpose
dfs(game, depth, path, visited)	Performs Depth-Limited Search (DLS) at the given depth
if depth == 0:	Stops further exploration when depth limit is reached
visited.add(state)	Keeps track of visited states to avoid loops
visited.remove(state)	Backtracks when a path is not successful
solver_ids(game_map)	Calls DFS repeatedly with increasing depth
depth += 1	Expands the search limit when no solution is found

In [1901]:
def dfs(game, depth, path, visited):
    if game.is_game_won():
        return path, len(visited)
    
    if depth == 0:
        return None, len(visited)
    
    directions = ['U', 'D', 'L', 'R']
    
    for direction in directions:
        new_game = Game(game.get_map())
        if new_game.apply_move(direction):
            state = (tuple(new_game.get_box_locations()), new_game.get_player_position())
            if state not in visited:
                visited.add(state)
                new_path = path + direction
                result, visited_count = dfs(new_game, depth - 1, new_path, visited)
                if result is not None:
                    return result, visited_count
                visited.remove(state)  
    
    return None, len(visited)


def solver_ids(game_map):
    game = Game(game_map)
    depth = 0
    visited = set()
    
    while True:
        visited.clear()
        visited.add((tuple(game.get_box_locations()), game.get_player_position()))  
        result, visited_count = dfs(game, depth, '', visited)
        
        if result is not None:
            return result, visited_count
        
        depth += 1  


### A*

### A* Algorithm Explanation*
The A (A-star) algorithm* is a widely used search algorithm that finds the shortest path from a starting state to a goal state. It is an improvement over Dijkstra's algorithm by incorporating heuristics to guide the search efficiently.
🔹 How Does A Work?*
A* maintains a priority queue (min-heap) where each state is evaluated using a cost function:
$$f(n)=g(n)+h(n)$$
g(n) = The cost to reach node n from the start state.
h(n) = The estimated cost (heuristic) from n to the goal.
The algorithm expands the most promising state (lowest 
f(n)) first, ensuring an optimal and efficient search.
🔹 Steps of A*
1.Initialize the priority queue (pq) with the start state and cost f=0.
2.Expand the state with the lowest f(n).
3.Check if the goal is reached (all boxes in goal positions).
If yes, return the path.
4.Generate all possible next states (moving in four directions).
Calculate g(n) and h(n).
Compute $$f(n)=g(n)+h(n)$$ and add the state to the queue.
5.Repeat until a solution is found or no states remain.

🔹Why Use A?*
✔ Guarantees the shortest solution (if h(n) is admissible).
✔ Efficient compared to uninformed searches (like BFS or DFS).
✔ Uses heuristics to avoid unnecessary paths.

🔹Heuristic Functions (h(n))
  A heuristic function estimates the remaining cost to reach the goal.
  We compare two heuristics used in our A* implementation:
  $$h(n)=∑∣B 
x
​
 −G 
x
​
 ∣+∣B 
y
​
 −G 
y
​
 ∣$$

  Where:
  Bx , By are the coordinates of a box.
  Gx , Gy are the coordinates of the nearest goal.
  We sum the Manhattan distance for all boxes.

  
  ✅ Advantages:
  ✔ Fast to compute.
  ✔ Guarantees an admissible heuristic.
  ✔ Works well in grid-based environments.

  ❌ Disadvantages:

  Ignores obstacles (walls, other boxes).
  Might underestimate the actual cost in complex environments.

2️⃣ Advanced Heuristic (Custom)
  This heuristic improves Manhattan Distance by considering:

  Each box's nearest goal (similar to Manhattan).
  Player's distance to the nearest box (to ensure movement efficiency).
  $$h(n)= 
each box
∑
​
 min(Manhattan distance to any goal)+min(Manhattan distance from player to any box)$$

 ✅ Advantages:
  ✔ More accurate than Manhattan Distance.
  ✔ Prioritizes states where the player is closer to an actionable box.

  ❌ Disadvantages:

  More computationally expensive than Manhattan.
  Still doesn’t account for obstacles like walls.



### Comparison of Heuristics in A*

The A* algorithm relies on heuristics to estimate the remaining cost to the goal. Below is a comparison of two heuristics:

| Heuristic        | Speed   | Accuracy        | Considers Player? | Considers Obstacles? |
|------------------|---------|----------------|--------------------|----------------------|
| **Manhattan**   | ✅ Fast  | ❌ Less accurate | ❌ No               | ❌ No               |
| **Advanced**    | ❌ Slower | ✅ More accurate | ✅ Yes              | ❌ No               |

**Which one is better?**  
- **Manhattan Distance** is fast but less accurate.  
- **Advanced Heuristic** is more precise but computationally expensive.  
- Neither heuristic considers obstacles, which can lead to overestimation.


### A* Heuristic Comparison

The A* algorithm uses heuristics to estimate the remaining cost to the goal. Below is a comparison of two heuristics:

| Heuristic        | Speed   | Accuracy        | Considers Player? | Considers Obstacles? |
|------------------|---------|----------------|--------------------|----------------------|
| **Manhattan**   | ✅ Fast  | ❌ Less accurate | ❌ No               | ❌ No               |
| **Advanced**    | ❌ Slower | ✅ More accurate | ✅ Yes              | ❌ No               |

#### **Which One is Better?**
- **Manhattan Distance** is faster but less accurate.  
- **Advanced Heuristic** is more precise but computationally expensive.  
- Neither heuristic considers obstacles, which can lead to overestimation.


In [1902]:
from heapq import heappop, heappush

def advanced_heuristic(state, game):
    _, boxes = state
    goals = game.get_goal_locations()
    player = game.get_player_position()
    
    total_cost = 0

    for bx, by in boxes:
        min_dist = float('inf')
        for gx, gy in goals:
            if not game.is_wall((gx, gy)):  
                min_dist = min(min_dist, abs(bx - gx) + abs(by - gy))
        
        total_cost += min_dist if min_dist != float('inf') else 1000  

    player_to_box_distance = min(abs(player[0] - bx) + abs(player[1] - by) for bx, by in boxes)

    return total_cost + player_to_box_distance


def solver_astar(game_map, heuristic_type="manhattan"):
    game = Game(game_map)
    initial_state = (game.get_player_position(), tuple(game.get_box_locations()))
    pq = []  
    visited = set()  

    heappush(pq, (0, 0, initial_state, ""))  
    
    while pq:
        f, g, (player_pos, boxes), path = heappop(pq)

        temp_game = Game(game_map)
        temp_game.set_player_position(player_pos)
        temp_game.set_box_positions(boxes)

        if temp_game.is_game_won():
            return path, len(visited)  

        state = (player_pos, boxes)
        if state in visited:
            continue
        visited.add(state)

        for direction in ['U', 'D', 'L', 'R']:
            new_game = Game(game_map)
            new_game.set_player_position(player_pos)
            new_game.set_box_positions(boxes)

            if new_game.apply_move(direction):  
                new_state = (new_game.get_player_position(), tuple(new_game.get_box_locations()))
                new_g = g + 1
                
                if heuristic_type == "advanced":
                    new_h = advanced_heuristic(new_state, new_game)
                elif heuristic_type == "manhattan":
                    new_h = sum(abs(bx - gx) + abs(by - gy) for bx, by in new_state[1] for gx, gy in new_game.get_goal_locations())
                else:
                    new_h = 0  
                
                new_f = new_g + new_h
                heappush(pq, (new_f, new_g, new_state, path + direction))

    return "No solution", len(visited)

## find Weighted A*

Explanation of solver_weighted_astar Algorithm
The function solver_weighted_astar implements the Weighted A* search algorithm, which is a variation of the standard A* algorithm. The key difference is that it introduces a weight factor w, which increases the influence of the heuristic function h(n) in the evaluation function:
$$f(n)=g(n)+w×h(n)$$
f(n)=g(n)+w×h(n)
Where:

𝑔
(
𝑛
)
g(n) = Actual cost from the start state to state 
𝑛
n (number of moves taken).
ℎ
(
𝑛
)
h(n) = Estimated heuristic cost from state 
𝑛
n to the goal.
𝑤
w = Weight that amplifies the heuristic’s importance.
This algorithm prioritizes states with the lowest 
𝑓
(
𝑛
)
f(n) value by using a priority queue (min-heap). The higher the weight 
𝑤
w, the more aggressively the algorithm favors heuristic estimates over actual cost.
✅ Impact of Changing 
𝑤
w (Weight Parameter)
The weight 
𝑤
w determines how much the search prioritizes heuristic estimates over actual cost:

🔹 If 
𝑤
=
1
w=1 (Standard A)*
The algorithm behaves like regular A* search.
It finds the optimal solution but may take longer.
🔹 If 
𝑤
>
1
w>1 (More Heuristic Influence)
The search is more greedy and prioritizes estimated paths over exact costs.
Advantages:
✔️ Runs faster because it aggressively follows heuristic estimates.
✔️ Reduces the number of visited states.
Disadvantages:
❌ May return suboptimal solutions (paths that are longer but appear promising based on heuristics).
🔹 If 
𝑤
<
1
w<1 (Less Heuristic Influence)
The heuristic has less effect, making the algorithm behave more like Uniform Cost Search (Dijkstra’s Algorithm).
Advantages:
✔️ Guarantees optimal solutions.
Disadvantages:
❌ Becomes very slow because it explores many unnecessary states.


from IPython.display import display, Markdown

weighted_a_summary = """
## ✅ **Impact of Different \( w \) Values in Weighted A***

| **\( w \) Value** | **Search Behavior** | **Speed** | **Solution Quality** |
|--------------|-------------------|------------|-------------------|
| **\( w = 1 \)** | Standard A* (balanced search) | Medium | ✅ Always optimal |
| **\( w > 1 \)** | More greedy, follows heuristic | 🚀 Faster | ❌ May be suboptimal |
| **\( w < 1 \)** | More uniform-cost-like | 🐢 Slower | ✅ Always optimal |

### **Key Takeaways:**  
✔ **Weighted A*** is a trade-off between **speed and solution quality**.  
✔ A **higher \( w \)** makes the algorithm faster but less optimal.  
✔ A **lower \( w \)** ensures optimal solutions but increases runtime.  
✔ The best \( w \) value **depends on the problem complexity**—usually **1.2 to 2** provides a good balance.  

🚀 **In practice, tuning \( w \) helps balance search efficiency vs. solution quality, making Weighted A* useful in real-time pathfinding and AI applications.**
"""

display(Markdown(weighted_a_summary))



✅ Final Takeaways
✔ Weighted A* is a trade-off between speed and solution quality.
✔ A higher 
𝑤
w makes the algorithm faster but less optimal.
✔ A lower 
𝑤
w ensures optimal solutions but increases runtime.
✔ The best 
𝑤
w value depends on the problem complexity—usually 1.2 to 2 provides a good balance.

🚀 In practice, tuning 
𝑤
w helps balance search efficiency vs. solution quality, making Weighted A useful in real-time pathfinding and AI applications.*

In [1903]:
from heapq import heappop, heappush

def advanced_heuristic(state, game):
    _, boxes = state
    goals = game.get_goal_locations()
    player = game.get_player_position()
    
    total_cost = 0
    for bx, by in boxes:
        min_dist = float('inf')
        for gx, gy in goals:
            if not game.is_wall((gx, gy)):  
                min_dist = min(min_dist, abs(bx - gx) + abs(by - gy))
        total_cost += min_dist if min_dist != float('inf') else 1000  
    
    player_to_box_distance = min(abs(player[0] - bx) + abs(player[1] - by) for bx, by in boxes)
    return total_cost + player_to_box_distance


def solver_weighted_astar(game_map, w=5):
    game = Game(game_map)
    initial_state = (game.get_player_position(), tuple(game.get_box_locations()))
    pq = []  
    visited = set()  
    
    heappush(pq, (0, 0, initial_state, ""))  
    
    while pq:
        f, g, (player_pos, boxes), path = heappop(pq)
        
        temp_game = Game(game_map)
        temp_game.set_player_position(player_pos)
        temp_game.set_box_positions(boxes)
        
        if temp_game.is_game_won():
            return path, len(visited)  
        
        state = (player_pos, tuple(boxes)) 
        if state in visited:
            continue
        visited.add(state)
        
        for direction in ['U', 'D', 'L', 'R']:
            new_game = Game(game_map)
            new_game.set_player_position(player_pos)
            new_game.set_box_positions(boxes)
            
            if new_game.apply_move(direction):  
                new_state = (new_game.get_player_position(), tuple(new_game.get_box_locations()))
                new_g = g + 1
                new_h = advanced_heuristic(new_state, new_game)
                new_f = new_g + w * new_h  
                heappush(pq, (new_f, new_g, new_state, path + direction))

    return "", len(visited)  

##weighted_astar

## Solve

In [1904]:
SOLVERS = {
    "BFS": solver_bfs,
    "DFS": solver_dfs,
    "IDS": solver_ids,
    "A*": solver_astar,
    "weighted_a" : solver_weighted_astar
}

In [1905]:
def solve(map, method):  
    
    if not is_valid_input(map, map_file_indices, method, SOLVERS):
        return
    
    file_name = "map" + str(map) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    
    start_time = time.time()
    moves, numof_visited_states = SOLVERS[method](game_map)
    end_time = time.time()
    print(f"{method} took {round(end_time - start_time, 2)} seconds on map {map} and visited {numof_visited_states} states.")
    
    if moves is None:
        print("No Solution Found!")
    else:
        print(f"{len(moves)} moves were used: {moves}")
            

###TESTS

In [1906]:
solve(1,"BFS")
solve(1, "DFS")
solve(1, "IDS")
solve(1,"A*")
solve(1,"weighted_a")

BFS took 0.01 seconds on map 1 and visited 44 states.
7 moves were used: UDDULRR
DFS took 0.0 seconds on map 1 and visited 7 states.
7 moves were used: RLLRDUU
IDS took 0.04 seconds on map 1 and visited 8 states.
7 moves were used: UDDULRR
A* took 0.01 seconds on map 1 and visited 43 states.
7 moves were used: DULRRLU
weighted_a took 0.0 seconds on map 1 and visited 13 states.
7 moves were used: UDLRRLD


from IPython.display import display, Markdown

results = """
## ✅ **Algorithm Performance on Map 1**

| **Algorithm**  | **Time Taken (seconds)** | **States Visited** | **Moves Used** |
|---------------|---------------------|----------------|---------------|
| **BFS**       | 0.02                 | 44             | UDDULRR       |
| **DFS**       | 0.00                 | 7              | RLLRDUU       |
| **IDS**       | 0.05                 | 8              | UDDULRR       |
| **A***        | 0.02                 | 43             | DULRRLU       |
| **Weighted A*** | 0.01               | 13             | UDLRRLD       |

✅ **Weighted A*** visited significantly fewer states while maintaining an optimal solution time.
"""

display(Markdown(results))


In [1907]:
solve(2,"BFS")
solve(2, "DFS")
solve(2,"IDS")
solve(2,"A*")
solve(2,"weighted_a")

BFS took 0.0 seconds on map 2 and visited 21 states.
6 moves were used: LUDDUL
DFS took 0.0 seconds on map 2 and visited 6 states.
6 moves were used: LLRDUU
IDS took 0.01 seconds on map 2 and visited 7 states.
6 moves were used: LUDDUL
A* took 0.01 seconds on map 2 and visited 23 states.
6 moves were used: LDULRU
weighted_a took 0.0 seconds on map 2 and visited 9 states.
6 moves were used: LUDLRD


from IPython.display import display, Markdown

results_map2 = """
## ✅ **Algorithm Performance on Map 2**

| **Algorithm**  | **Time Taken (seconds)** | **States Visited** | **Moves Used** |
|---------------|---------------------|----------------|---------------|
| **BFS**       | 0.00                 | 21             | LUDDUL        |
| **DFS**       | 0.00                 | 6              | LLRDUU        |
| **IDS**       | 0.01                 | 7              | LUDDUL        |
| **A***        | 0.01                 | 23             | LDULRU        |
| **Weighted A*** | 0.00               | 9              | LUDLRD        |

✅ **Weighted A*** visited fewer states and performed efficiently while maintaining a correct solution.
"""

display(Markdown(results_map2))


In [1908]:
solve(3,"BFS")
solve(3, "DFS")
solve(3,"IDS")
solve(3,"A*")
solve(3,"weighted_a")

BFS took 0.02 seconds on map 3 and visited 122 states.
13 moves were used: ULDDUUUURDDDD
DFS took 0.01 seconds on map 3 and visited 13 states.
13 moves were used: ULDDUUUURDDDD
IDS took 0.16 seconds on map 3 and visited 14 states.
13 moves were used: ULDDUUUURDDDD
A* took 0.02 seconds on map 3 and visited 86 states.
14 moves were used: ULUURDDDDUULDD
weighted_a took 0.0 seconds on map 3 and visited 18 states.
13 moves were used: ULDDUUUURDDDD


from IPython.display import display, Markdown

results_map3 = """
## ✅ **Algorithm Performance on Map 3**

| **Algorithm**  | **Time Taken (seconds)** | **States Visited** | **Moves Used** |
|---------------|---------------------|----------------|---------------|
| **BFS**       | 0.02                 | 122            | ULDDUUUURDDDD  |
| **DFS**       | 0.01                 | 13             | ULDDUUUURDDDD  |
| **IDS**       | 0.18                 | 14             | ULDDUUUURDDDD  |
| **A***        | 0.02                 | 86             | ULUURDDDDUULDD  |
| **Weighted A*** | 0.00               | 18             | ULDDUUUURDDDD  |

✅ **Weighted A*** visited significantly fewer states compared to BFS and A*, making it more efficient while still finding an optimal solution.
"""

display(Markdown(results_map3))


In [1915]:
solve(4,"BFS")
solve(4, "DFS")
solve(4,"IDS")
solve(4,"A*")
solve(4,"weighted_a")

BFS took 0.0 seconds on map 4 and visited 2 states.
11 moves were used: No solution
DFS took 0.0 seconds on map 4 and visited 2 states.
11 moves were used: No solution


KeyboardInterrupt: 

In [1914]:
solve(5,"BFS")
solve(5, "DFS")
solve(5,"IDS")
solve(5,"A*")
solve(5,"weighted_a")

BFS took 0.99 seconds on map 5 and visited 6328 states.
15 moves were used: ULDDRDLLLUUURUL
DFS took 0.41 seconds on map 5 and visited 51 states.
51 moves were used: LRDLLLLUURRDRRDLLRRDLLLLURRLURRRDLLLRRRULLLLDRUURUL
IDS took 31.08 seconds on map 5 and visited 16 states.
15 moves were used: ULDDRDLLLUUURUL
A* took 0.17 seconds on map 5 and visited 1212 states.
15 moves were used: LULDLRDRDLLULUU
weighted_a took 0.01 seconds on map 5 and visited 46 states.
17 moves were used: LULDLDLUURRDDRDLL


from IPython.display import display, Markdown

results_map5 = """
## ✅ **Algorithm Performance on Map 5**

| **Algorithm**  | **Time Taken (seconds)** | **States Visited** | **Moves Used** |
|---------------|---------------------|----------------|---------------|
| **BFS**       | 0.99                 | 6328           | ULDDRDLLLUUURUL  |
| **DFS**       | 0.41                 | 51             | LRDLLLLUURRDRRDLLRRDLLLLURRLURRRDLLLRRRULLLLDRUURUL  |
| **IDS**       | 31.08                | 16             | ULDDRDLLLUUURUL  |
| **A***        | 0.17                 | 1212           | LULDLRDRDLLULUU  |
| **Weighted A*** | 0.01               | 46             | LULDLDLUURRDDRDLL  |

✅ **Weighted A*** visited significantly fewer states compared to BFS and A*, making it the fastest approach. However, it found a 17-move solution instead of the optimal 15-move solution.
"""

display(Markdown(results_map5))


In [1916]:
solve(6,"BFS")
solve(6,"A*")
solve(6,"weighted_a")

BFS took 10.18 seconds on map 6 and visited 17807 states.
34 moves were used: UUUUURRRLLLLLLLDDDDDDDDDRDLRRRRRRR
A* took 13.7 seconds on map 6 and visited 17579 states.
34 moves were used: DDDDDRRRLLLLLLLRUUUUUUUUUULRRRRRRR
weighted_a took 0.46 seconds on map 6 and visited 579 states.
34 moves were used: UUUUURRRLLLLLLLDDDDDDDDDRDLRRRRRRR


from IPython.display import display, Markdown

results_map6 = """
## ✅ **Algorithm Performance on Map 6**

| **Algorithm**  | **Time Taken (seconds)** | **States Visited** | **Moves Used** |
|---------------|---------------------|----------------|---------------|
| **BFS**       | 10.18                | 17807          | UUUUURRRLLLLLLLDDDDDDDDDRDLRRRRRRR  |
| **A***        | 13.70                | 17579          | DDDDDRRRLLLLLLLRUUUUUUUUUULRRRRRRR  |
| **Weighted A*** | 0.46               | 579            | UUUUURRRLLLLLLLDDDDDDDDDRDLRRRRRRR  |

✅ **Weighted A*** dramatically reduced the number of states visited (579 vs. 17,807 in BFS), making it much faster while still finding the same optimal solution.
"""

display(Markdown(results_map6))


In [1917]:
solve(7,"BFS")
solve(7,"A*")
solve(7,"weighted_a")

BFS took 185.11 seconds on map 7 and visited 644754 states.
34 moves were used: RURRDDDDLDRUUUULLLRDRDRDDLLDLLUUDR
A* took 272.13 seconds on map 7 and visited 692479 states.
34 moves were used: RURRDDDDLDRUUUULLLRDRDRDDLLDLLURLU
weighted_a took 0.79 seconds on map 7 and visited 1820 states.
38 moves were used: RURRDLLLRRRDDDLDRUUUULLDRDRDDLLDLLUUDR


from IPython.display import display, Markdown

results_map7 = """
## ✅ **Algorithm Performance on Map 7**

| **Algorithm**  | **Time Taken (seconds)** | **States Visited** | **Moves Used** |
|---------------|---------------------|----------------|---------------|
| **BFS**       | 185.11               | 644754         | RURRDDDDLDRUUUULLLRDRDRDDLLDLLUUDR  |
| **A***        | 272.13               | 692479         | RURRDDDDLDRUUUULLLRDRDRDDLLDLLURLU  |
| **Weighted A*** | 0.79                | 1820           | RURRDLLLRRRDDDLDRUUUULLDRDRDDLLDLLUUDR  |

✅ **Weighted A*** dramatically reduced search time and the number of visited states (only 1820 vs. 644,754 in BFS), but at the cost of finding a slightly longer solution (38 moves instead of 34).
"""

display(Markdown(results_map7))


In [1918]:
solve(8,"BFS")
solve(8,"IDS")
solve(8,"A*")
solve(8,"weighted_a")

BFS took 4.96 seconds on map 8 and visited 7419 states.
14 moves were used: UURDLDRRDRURDR
IDS took 181.82 seconds on map 8 and visited 15 states.
14 moves were used: UURDLDRRDRURDR
A* took 0.05 seconds on map 8 and visited 72 states.
19 moves were used: URRRRRRRRRRRRRURDDR
weighted_a took 0.09 seconds on map 8 and visited 121 states.
19 moves were used: URRRRRRRRRRRRRURDDR


from IPython.display import display, Markdown

results_map8 = """
## ✅ **Algorithm Performance on Map 8**

| **Algorithm**  | **Time Taken (seconds)** | **States Visited** | **Moves Used** |
|---------------|---------------------|----------------|---------------|
| **BFS**       | 4.96                 | 7419           | UURDLDRRDRURDR  |
| **IDS**       | 181.82               | 15             | UURDLDRRDRURDR  |
| **A***        | 0.05                 | 72             | URRRRRRRRRRRRRURDDR  |
| **Weighted A*** | 0.09               | 121            | URRRRRRRRRRRRRURDDR  |

✅ **A* and Weighted A*** performed significantly faster than BFS and IDS. However, they found a **longer solution (19 moves instead of 14)**.
"""

display(Markdown(results_map8))


In [1919]:
solve(10,"BFS")
solve(10,"A*")
solve(10,"weighted_a")

BFS took 140.4 seconds on map 10 and visited 241503 states.
46 moves were used: RRRRRDRULURULLLULDRUUULDRDLDRRDRULURURDDRDLLLL
A* took 112.98 seconds on map 10 and visited 151607 states.
46 moves were used: RRRRRDRULURULLLDLLURUULDRDLDRRDRULURURDDRDLLLL
weighted_a took 13.52 seconds on map 10 and visited 18931 states.
46 moves were used: RRRRRDRULURULLLULDRUUULDRDLDRRDRULURURDDRDLLLL


from IPython.display import display, Markdown

results_map10 = """
## ✅ **Algorithm Performance on Map 10**

| **Algorithm**  | **Time Taken (seconds)** | **States Visited** | **Moves Used** |
|---------------|---------------------|----------------|---------------|
| **BFS**       | 140.4                | 241503         | RRRRRDRULURULLLULDRUUULDRDLDRRDRULURURDDRDLLLL  |
| **A***        | 112.98               | 151607         | RRRRRDRULURULLLDLLURUULDRDLDRRDRULURURDDRDLLLL  |
| **Weighted A*** | 13.52               | 18931          | RRRRRDRULURULLLULDRUUULDRDLDRRDRULURURDDRDLLLL  |

✅ **Weighted A*** significantly reduced the number of visited states and execution time compared to BFS and A*, while still finding an optimal solution.
"""

display(Markdown(results_map10))


In [ ]:
def solve_all():
    for map in range(min(map_file_indices), max(map_file_indices) + 1):
        for method in SOLVERS.keys():
            solve(map, method)
            

In [ ]:
game_map_string =  """  W   P1  W   .   .   W   W
W   H   W   .   B2  W   W
W   .   W   .   .   W   W
W   G2  W   B1  P1  W   W
W   W   W   .   W   W   W
W   W   W   G1  W   W   W
W   W   W   W   W   W   W
"""  
game = Game(game_map_string)  # مقداردهی اولیه با نقشه

solution = "ULUURDDDDUULDD"  # راه‌حل پیشنهادی
valid = game.is_solution_valid(solution)

if valid:
    print("yes")  # مسیر معتبر است
else:
    print("no")   # مسیر نامعتبر است

if game.is_game_won():
    print("Game is won!")
else:
    print("Game is not won yet.")



yes
Game is won!


## Quetions 1
State Definition in This Problem
In this problem, the state represents the configuration of the warehouse at any given moment. The most basic definition of a state includes:

Player's Position (H) → The current location of the player.
Boxes’ Positions (B1, B2, ... Bn) → The coordinates of all movable boxes.
Thus, a state can be represented as:

state
=
(
player position
,
tuple of box positions
)
state=(player position,tuple of box positions)
Since the walls (W) and goal positions (G) are fixed, they do not need to be included in the state representation.

Naïve State Representation: The Entire Warehouse Map
A straightforward approach is to define a state as the entire warehouse map at a given moment, where each cell explicitly contains its contents (empty space, walls, player, boxes, etc.).
Issues with the Naïve Representation & How to Improve It
1. Large State Space & Redundancy
The map has many static elements that do not change between states (walls, goals).
Storing and comparing entire maps in the search algorithm increases memory usage and slows down state comparisons.
✅ Improvement:

Instead of storing the entire map, define a state using only dynamic elements:
state
=
(
player position
,
tuple of box positions
)
state=(player position,tuple of box positions)
This significantly reduces memory usage and speeds up state comparison.
2. Inefficient State Comparison
Comparing entire maps requires checking each cell, which is computationally expensive.
✅ Improvement:

By using tuples of coordinates (which are immutable and hashable), states can be stored efficiently in a hash set (for fast lookup in search algorithms like A*).
3. Lack of Symmetry Reduction
Some states may be equivalent but appear different in a full map representation.
Example: If the player moves around a box without changing its position, the overall situation hasn’t changed.
✅ Improvement:

We can ignore player position when checking for repeated states, unless it affects possible moves.
Another approach is to use canonical representations, where states that are functionally equivalent are treated as identical.
4. Deadlocks Are Not Considered
The naïve state representation does not check if a box is pushed into a deadlock position (e.g., against a corner where it can’t be moved).
This leads to unnecessary exploration of impossible solutions.
✅ Improvement:

Add a deadlock detection mechanism:
If a box is against a wall corner and not on a goal, the state should be considered invalid.
Identify freezing positions where a box can no longer be moved, eliminating them early in the search.
Final Optimized State Representation
A well-optimized state representation should include:

Player Position → Important for movement but ignored in equivalent states.
Boxes’ Positions (Tuple) → Immutable and used for fast lookup.
Deadlock Prevention Check → Avoids searching impossible paths.
Optimized State Definition:

state
=
(
player position
,
tuple of box positions
)
state=(player position,tuple of box positions)
Extra Optimizations:

Store states in a hash table for fast lookup.
Use symmetry reduction to avoid redundant states.
Implement deadlock detection to prune useless branches.
This optimized definition makes algorithms like A* and Weighted A* much more efficient, reducing both memory usage and computational time.
"The problem with this definition is that it stores static elements of the map (such as walls and goals) in every state, increasing the state space, slowing down the search, and causing unnecessary comparisons."






## Quetions 2
✅ Optimized State Definition
Instead of storing the entire map, we define the state using only the dynamic elements:
state=(player position,tuple of box positions)
Why?

Walls (W) and goals (G) are static and do not need to be stored in every state.
Using a tuple for boxes (tuple of box positions) makes states hashable and enables fast lookups.
This reduces the state space, speeds up comparisons, and makes the search more efficient.
✅ Defining Actions Based on This State
Each action represents a move by the player (H), which may also push a box. The possible actions include:

Normal movement (U, D, L, R):

If the next cell is empty (.), the player can move there.
Pushing a box (Push):

If the next cell contains a box (B) and the cell after it is empty (.) or a portal (P), the player can push the box.
The new state updates both the player's position and the new box positions accordingly.
✅ Action rules (in formula form):
If the player H is at (px, py) and wants to move in direction (dx, dy):

Normal move:
Allowed if (px + dx, py + dy) is empty.
Pushing a box:
Allowed if (px + dx, py + dy) contains a box and (px + 2dx, py + 2dy) is empty or a portal.
💡 Handling Portals:

If a player or a box enters a portal, it is teleported to the corresponding portal exit, and the state is updated accordingly.



## Quetions 3

Formal Problem Definition Based on Our State Representation
In this problem, we define state, actions, initial state, and goal state based on the optimized representation where:
state=(player position,tuple of box positions)
✅ 1. State Definition
A state represents a valid configuration of the warehouse at a given moment.
state=(player position,tuple of box positions)
Where:

Player position: (x, y) coordinate of the player.
Box positions: A tuple containing the coordinates of all boxes (bx, by).
The walls (W) and goals (G) are fixed and do not need to be stored in each state.

✅ 2. Initial State
The initial state is the configuration of the warehouse at the beginning of the game. It consists of:

The starting position of the player.
The initial positions of all boxes.
Example Initial State Representation:
initial state=(player’s start position,tuple of initial box positions)

✅ 3. Goal State
A goal state is reached when all boxes are on their designated goal positions.

Mathematical Representation:

$$∀
𝑖
,
(
𝐵
𝑖
∈
𝐺
𝑖
)$$
Meaning that for every box Bi
​
 , its position must match its corresponding goal Gi.
✅ 4. Action Definition
An action represents a movement the player can make, either moving freely or pushing a box.

Possible Actions
Move (U, D, L, R):

The player moves one step in a direction if the next cell is empty (.) or a portal (P).
Push a Box:

If the player moves into a box and the cell after the box is empty (.) or a portal (P), the box is pushed.
Formal Action Representation:
If the player is at (px, py), moving in direction (dx, dy):

Valid move:
if 
$$(
𝑝
𝑥
+
𝑑
𝑥
,
𝑝
𝑦
+
𝑑
𝑦
)$$
 is empty, move is allowed.
if (px+dx,py+dy) is empty, move is allowed.
Valid push:
if 
$$(
𝑝
𝑥
+
𝑑
𝑥
,
𝑝
𝑦
+
𝑑
𝑦
)$$
 contains a box and 
$$(
𝑝
𝑥
+
2
𝑑
𝑥
,
𝑝
𝑦
+
2
𝑑
𝑦
)$$
 is empty, push is allowed.
if (px+dx,py+dy) contains a box and (px+2dx,py+2dy) is empty, push is allowed.

✅ 5. Transition Function (State Change)
The transition function updates the state based on an action.

state’
=
apply(action, state)
state’=apply(action, state)
Where:

If the player moves without pushing: The player's position updates.
If the player pushes a box: The box’s position updates along with the player’s.
If a portal is used: The new position is determined by portal connections.

✅ 6. Cost Function
The cost function assigns a cost to each action.

cost(action)
=
1
cost(action)=1
Each move has a uniform cost of 1.
The total cost of a path is the number of moves required to reach the goal.
✅ 7. Deadlock Detection (Pruning States)
Some states should be pruned if they are irreversible deadlocks:

If a box is stuck against a wall and not on a goal, the state should be ignored.
If a box is placed in a corner where it cannot move in any direction, it is a deadlock state.



from IPython.display import display, Markdown

summary = """
## ✅ Final Summary

| **Component**        | **Definition**  |
|---------------------|----------------|
| **State**          | `(player_position, tuple of box_positions)` |
| **Initial State**  | Starting positions of player and boxes |
| **Goal State**     | All boxes on their goal positions |
| **Actions**        | Move (U, D, L, R), Push (if allowed) |
| **Transition Function** | Updates state based on the player's move |
| **Cost Function**  | Each move has a cost of 1 |
| **Deadlock Detection** | Prunes states where boxes are stuck |

✅ **This formal definition optimizes state representation, improves search efficiency, and prevents unnecessary exploration of impossible paths.** 🚀
"""

display(Markdown(summary))


## Quetions 3
Optimizing the Search Space and Improving the Problem-Solving Approach
Solving this problem efficiently requires optimizing the search space and reducing unnecessary state expansions. Below, we explore various ways to improve the problem-solving approach and minimize redundant computations in search algorithms.

✅ 1. Reducing the Search Space
1.1. Avoid Expanding All Possible Moves
In naive search approaches, every state expands all possible moves (U, D, L, R) without considering whether they are useful. This results in:

Exponential growth in the search space.
Unnecessary exploration of bad or redundant paths.
✅ Optimization:

Only consider moves that lead to meaningful progress (e.g., pushing a box toward a goal).
Avoid moves that simply move the player in circles without affecting the game state.
1.2. Detect and Prune Deadlock States
Some states lead to irreversible deadlocks, where a box is placed in a position where it can no longer reach a goal.

✅ Optimization:

If a box is pushed into a corner where it has no way to escape, eliminate this state.
If a box is against a wall and not aligned with a goal, eliminate this state.
Implement pattern-based deadlock detection, where common losing patterns (like blocking a necessary path) are recognized early.
1.3. Identify Equivalent States (Symmetry Reduction)
Many states are functionally identical even if they look different.

✅ Optimization:

If two states have the same box positions but different player positions, consider them equivalent in some cases.
Store only unique box configurations in the visited states list to prevent redundant expansions.
✅ 2. Improving Heuristic Functions in Informed Search (A & Weighted A)**
2.1. Using a Stronger Heuristic
A good heuristic helps the algorithm explore promising paths first and avoid wasting time on unhelpful branches.

✅ Better Heuristic for A*:
$$h(n)=∑Manhattan distance of each box to its nearest goal+Player distance to nearest box$$
This ensures:

Boxes move closer to their correct goals first.
The player reaches the most important box faster.
2.2. Weighted A for Faster Convergence*
Instead of using A* (f(n) = g(n) + h(n)), we can apply Weighted A*:
$$f(n)=g(n)+w×h(n)$$

✅ 3. Do We Need to Expand Every Possible Move?
No, Expanding Every Move is Wasteful!
Expanding all possible moves without filtering useful ones leads to slow search times and unnecessary memory usage.

Consequences of Expanding Every Possible Move:
❌ Wastes time searching bad paths
❌ Explores meaningless movements (e.g., moving back and forth without affecting boxes)
❌ Increases memory consumption and algorithm complexity

✅ Better Strategy:

Expand only promising moves (e.g., moving towards unaligned boxes).
Use priority-based pruning to avoid redundant expansions.
✅ 4. Additional Considerations & Actions to Improve Efficiency
4.1. Using Iterative Deepening in Uninformed Search (IDS)
Instead of BFS (which stores all nodes), use Iterative Deepening Search (IDS) to combine the depth-limiting of DFS with the completeness of BFS.
4.2. Multi-Goal Pathfinding
Instead of solving the problem box-by-box, consider the global arrangement of all boxes to prioritize the best sequence of movements.
4.3. Efficient State Storage (Hashing)
Store visited states using hash maps instead of lists for faster lookup and reducing redundant states.



from IPython.display import display, Markdown

summary = """
## ✅ Final Summary of Improvements

| **Optimization**                 | **Effect**  |
|---------------------------------|-------------|
| **Prune unnecessary moves** | Reduces redundant state expansion |
| **Deadlock detection** | Prevents exploring impossible states |
| **Use a strong heuristic** | Guides search more efficiently |
| **Weighted A*** | Balances speed vs. optimality |
| **Store unique box configurations** | Reduces memory usage |
| **Avoid back-and-forth movements** | Prevents looping in the search tree |
| **Use hashing for visited states** | Improves lookup efficiency |
| **Iterative Deepening Search (IDS) for uninformed search** | Saves memory compared to BFS |

✅ **By applying these techniques, we can significantly reduce the search space, improve execution speed, and find better solutions faster.** 🚀
"""

display(Markdown(summary))


## Quetions 4
Why Do We Still Use DFS Even Though It Is Not Optimal?
Depth-First Search (DFS) is not optimal because it does not guarantee finding the shortest or least-cost path. However, despite this limitation, DFS is still widely used in many scenarios due to its advantages in memory efficiency, speed in specific cases, and its ability to explore deep structures efficiently. Below are the key reasons why DFS remains useful:

✅ 1. Memory Efficiency (Low Space Complexity)
DFS has a space complexity of O(d), where d is the depth of the deepest node in the search tree.

Unlike Breadth-First Search (BFS), which requires storing all explored nodes (O(b^d)), DFS only needs to store the current path.
This makes DFS suitable for large problem spaces where storing all nodes is impractical.
✅ Example Use Case:

In problems with deep search trees (e.g., pathfinding in large mazes), BFS may run out of memory, but DFS can still explore efficiently.
✅ 2. Faster in Certain Scenarios
DFS can quickly find a solution if the solution is deep in the search tree.

If an answer is in a deeper level, DFS may reach it faster than BFS, which explores all shallow nodes first.
When combined with heuristics (e.g., iterative deepening DFS or depth-limited DFS), DFS can prioritize promising paths and improve efficiency.
✅ Example Use Case:

In backtracking algorithms (e.g., solving Sudoku, N-Queens), DFS is much more efficient because it eliminates invalid paths early.
✅ 3. Useful for Complete State Space Exploration
DFS explores all possible paths, making it useful in scenarios where full exploration is necessary.

Unlike BFS, which stops at the shortest solution, DFS fully explores a path before backtracking.
This is useful when we need all possible solutions, not just the shortest one.
✅ Example Use Case:

Generating all possible solutions (e.g., all valid sequences of moves in a game).
Finding connected components in graphs (e.g., detecting cycles in directed graphs).
✅ 4. DFS is Easier to Implement Recursively
DFS can be easily implemented using recursion, whereas BFS requires an explicit queue.

Recursive DFS takes advantage of the call stack, making it shorter and more readable than iterative BFS.
✅ Example Use Case:

Tree traversals (e.g., Inorder, Preorder, Postorder traversals of a binary tree).
✅ 5. DFS Variants Can Improve Performance
Although standard DFS is not optimal, its variants provide better performance while maintaining DFS’s efficiency.

from IPython.display import display, Markdown

dfs_variants = """
## 🔍 **Key DFS Variants**
| **DFS Variant** | **Improvement** |
|---------------|----------------|
| **Iterative Deepening DFS (IDDFS)** | Combines DFS’s low memory with BFS’s completeness |
| **Depth-Limited DFS** | Prevents infinite loops in deep search spaces |
| **Bidirectional Search** | Uses DFS in one direction and another search in reverse |
| **Heuristic DFS (Greedy DFS)** | Guides search using heuristics to prioritize promising paths |

✅ **IDDFS is commonly used in game AI and shortest path problems where BFS is too memory-intensive.**
"""
display(Markdown(dfs_variants))

✅ 6. DFS is Useful Even When Not the Best Choice
In many problems, DFS is not the best choice but is still good enough for practical applications.
If we don’t need the shortest path, DFS is often faster and simpler to implement than A* or BFS.
✅ Example Use Case:

If we only care about finding a solution (not the optimal one), DFS works well.


from IPython.display import display, Markdown

final_summary = """
## ✅ **Final Summary**
| **Reason** | **Why DFS is Still Used** |
|------------|---------------------------|
| **Low Memory Usage** | DFS stores fewer nodes compared to BFS, making it more scalable. |
| **Can Find Solutions Faster** | If the solution is deep, DFS may reach it before BFS. |
| **Explores All Solutions** | Useful for generating multiple solutions, not just the shortest one. |
| **Easier Recursive Implementation** | Recursive DFS is shorter and cleaner than BFS. |
| **DFS Variants Improve Efficiency** | IDDFS and Heuristic DFS improve performance while keeping DFS advantages. |
| **Works Well Even When Not Optimal** | Sometimes, finding any solution is enough, not necessarily the best one. |

✅ **Even though DFS is not optimal, it is still valuable due to its efficiency in memory, ability to explore deep structures, and versatility in different types of problems.** 🚀
"""
display(Markdown(final_summary))


How Does IDS Combine DFS and BFS, and What Problem Does It Solve?
Iterative Deepening Search (IDS) is a hybrid algorithm that combines the low memory usage of DFS with the completeness and optimality of BFS. It is designed to address the memory limitations of BFS while avoiding the incompleteness and inefficiency of DFS in deep or infinite search spaces.

✅ 1. Strengths and Weaknesses of DFS and BFS
🔹 Depth-First Search (DFS)
✔️ Low memory usage: Only stores the current path, requiring O(d) space, where d is the depth of the search.
✔️ Fast in deep solutions: If the solution is deep, DFS may reach it quickly.
❌ Not optimal: May return a longer path instead of the shortest one.
❌ Can get stuck in deep or infinite paths: DFS may keep going down an infinite branch and never find a solution.

🔹 Breadth-First Search (BFS)
✔️ Always finds the shortest path (if the cost is uniform): BFS guarantees optimality in unweighted graphs.
✔️ Complete: It will always find a solution if one exists.
❌ High memory usage: BFS stores all visited nodes, requiring O(b^d) space, where b is the branching factor.
❌ Slow in deep solutions: BFS explores all shallow nodes before going deeper, making it inefficient for deep solutions.

✅ 2. How IDS Works
IDS performs repeated DFS searches with an increasing depth limit. Each time, it expands the depth by one level until the solution is found.

Algorithm Steps:
Start with depth limit = 0.
Run DFS with the current depth limit.
If a solution is found, return it.
If not, increase the depth limit and repeat DFS.
Continue until the solution is found or all possibilities are explored.
This effectively simulates BFS because it explores nodes in increasing depth order, but without the memory overhead of BFS.

✅ 3. What Problem Does IDS Solve?
IDS is designed to solve the problem of high memory usage in BFS while maintaining completeness and optimality. It is especially useful when:

The solution is unknown and could be deep.
Memory constraints prevent using BFS.
We need an optimal solution but cannot afford BFS’s high space complexity.
🔹 Why is IDS Better Than DFS?
✅ DFS can get stuck in deep or infinite paths, while IDS resets the depth each time, preventing infinite loops.
✅ DFS is not optimal, but IDS, like BFS, finds the shortest path in uniform-cost problems.

🔹 Why is IDS Better Than BFS?
✅ BFS consumes too much memory (O(b^d)), while IDS only requires O(d) memory.
✅ IDS can be used in deep search spaces where BFS would be infeasible.


from IPython.display import display, Markdown

performance_analysis = """
## 🔍 **Performance Analysis of IDS**

| **Metric** | **DFS** | **BFS** | **IDS** |
|------------|--------|--------|--------|
| **Completeness** | ❌ No | ✅ Yes | ✅ Yes |
| **Optimality** | ❌ No | ✅ Yes (if uniform cost) | ✅ Yes (if uniform cost) |
| **Time Complexity** | **O(b^d)** | **O(b^d)** | **O(b^d)** |
| **Space Complexity** | **O(d)** | **O(b^d)** | **O(d)** |

🚀 **IDS achieves BFS’s completeness and optimality while using DFS’s low memory.**
"""

display(Markdown(performance_analysis))


✅ 5. When to Use IDS?
IDS is useful in cases where:

BFS is too memory-intensive.
DFS might go too deep and miss the solution.
We need both optimality and low memory usage.
✅ IDS is commonly used in AI search problems, game solvers, and scenarios where memory is a constraint.


from IPython.display import display, Markdown

final_summary = """
## ✅ **Final Summary**

| **Feature** | **DFS** | **BFS** | **IDS (Combination of DFS & BFS)** |
|------------|--------|--------|--------------------------------|
| **Memory Usage** | ✅ Low (O(d)) | ❌ High (O(b^d)) | ✅ Low (O(d)) |
| **Guarantees Optimality?** | ❌ No | ✅ Yes (if uniform cost) | ✅ Yes (if uniform cost) |
| **Completeness** | ❌ No (may go infinitely deep) | ✅ Yes | ✅ Yes |
| **Avoids Infinite Loops?** | ❌ No | ✅ Yes | ✅ Yes |
| **Best Used When** | Deep search space, no need for optimality | Shallow search space, need for shortest path | Large search space, need for optimality & low memory |

✅ **IDS is the best choice when memory efficiency and optimality are both required. It balances the advantages of DFS and BFS, making it one of the most effective search strategies in AI and graph traversal.** 🚀
"""

display(Markdown(final_summary))


✅ IDS is the best choice when memory efficiency and optimality are both required. It balances the advantages of DFS and BFS, making it one of the most effective search strategies in AI and graph traversal

## Quetions 7
Comparison of Heuristic Functions: Admissibility & Consistency
In this problem, two heuristic functions are used in the A* algorithm:

Manhattan Distance Heuristic:
$$h(n)=∑(Manhattan distance of each box to every goal)$$
Advanced Heuristic:
$$h(n)=∑(Min Manhattan distance from each box to a goal)+Player distance to nearest box$$
We analyze both heuristics for admissibility and consistency to determine their effectiveness in guiding the A* search algorithm.
✅ 1. Admissibility Analysis
A heuristic is admissible if it never overestimates the actual cost to reach the goal. In other words, it must always be ≤ the true cost of the optimal solution.

🔹 Manhattan Distance Heuristic
✔️ Admissible because:

Manhattan distance provides a lower bound on the actual path cost, since it only considers straight-line movement and ignores obstacles.
The real cost must be at least the sum of the box-to-goal distances, which this heuristic correctly estimates.
🔹 Advanced Heuristic
✔️ Admissible because:

It considers both box-to-goal distances and player-to-box distance, both of which are necessary moves.
The heuristic is always a lower bound on the real cost, as it does not account for obstacles or potential deadlocks.
✅ Conclusion:
Both heuristics are admissible, meaning they will not cause A* to overestimate costs and will always find an optimal solution if one exists.

✅ 2. Consistency Analysis (Monotonicity)
A heuristic is consistent (or monotonic) if:
$$h(n)≤c(n,n 
′
 )+h(n 
′
 )$$
 where:

ℎ
(
𝑛
)
h(n) is the heuristic value at state 
𝑛
n.
ℎ
(
𝑛
′
)
h(n 
′
 ) is the heuristic value at the successor state 
𝑛
′
n 
′
 .
𝑐
(
𝑛
,
𝑛
′
)
c(n,n 
′
 ) is the actual cost between 
𝑛
n and 
𝑛
′
n 
′
  (usually 1 in uniform-cost problems).
🔹 Manhattan Distance Heuristic
✔️ Consistent because:

Moving a box at most decreases its Manhattan distance by 1 per move.
Since each move costs at least 1, the heuristic difference never exceeds the real cost.
🔍 Example:
If a box moves one step toward its goal, 
ℎ
(
𝑛
′
)
h(n 
′
 ) decreases by at most 1, and since the move cost is also 1, the inequality 
ℎ
(
𝑛
)
≤
𝑐
(
𝑛
,
𝑛
′
)
+
ℎ
(
𝑛
′
)
h(n)≤c(n,n 
′
 )+h(n 
′
 ) holds.

🔹 Advanced Heuristic
❌ May be inconsistent because:

The player-to-box distance term can change unpredictably between states.
If the player moves toward a box, the heuristic might decrease more than the movement cost.
This could violate the consistency condition.
🔍 Example of inconsistency:

If the player moves closer to a box, the heuristic may decrease significantly (e.g., by 3), while the actual move cost is just 1, violating the consistency condition.
✅ Conclusion:

Manhattan heuristic is consistent.
Advanced heuristic may be inconsistent due to player movement affecting heuristic values non-linearly.



from IPython.display import display, Markdown

heuristic_comparison = """
## ✅ **Final Summary: Heuristic Comparison**

| **Heuristic** | **Admissible?** | **Consistent?** | **Pros** | **Cons** |
|--------------|--------------|--------------|------------------|------------------|
| **Manhattan Distance** | ✅ Yes | ✅ Yes | Guarantees optimality, simple, fast | Ignores player movement, does not account for obstacles |
| **Advanced Heuristic** | ✅ Yes | ❌ May be inconsistent | More informed (accounts for player movement) | May cause A* to re-expand nodes due to inconsistency |

### **Key Takeaways:**  
✅ **Manhattan Distance is both admissible and consistent**, making it ideal for A* as it guarantees optimality.  
⚠️ **Advanced Heuristic is admissible but may be inconsistent**, meaning A* might need to re-expand nodes, making it slightly less efficient.  

🚀 **If using the advanced heuristic, implementing an "inconsistency correction" (like a closed list re-check) can improve stability.**
"""

display(Markdown(heuristic_comparison))


Key Takeaways:
✅ Manhattan Distance is both admissible and consistent, making it ideal for A* as it guarantees optimality.
⚠️ Advanced Heuristic is admissible but may be inconsistent, meaning A* might need to re-expand nodes, making it slightly less efficient.

🚀 If using the advanced heuristic, implementing an "inconsistency correction" (like a closed list re-check) can improve stability